In [1]:
%pip install numpy pandas matplotlib seaborn xlrd scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
data = pd.read_csv(r"C:\Users\maroo\Downloads\UCI_Credit_Card.csv")

In [4]:
data_copy = data.copy()

In [5]:
#since the -1 and -2 hold some information value of non-default behaviour we create dummy variables for that
import numpy as np
for variable in data_copy.columns[6:12]:
    data_copy[variable+"_no_card_use"] = np.where(-2, 1, 0)
    data_copy[variable+"_payed_off"] = np.where(-1, 1, 0)
    data_copy[variable] = np.where(data_copy[variable] < 0, 0, data_copy[variable])

In [6]:
cont_variables = [
    'LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
    'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5',
    'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5',
    'PAY_AMT6', 'PAY_0_no_card_use', 'PAY_0_payed_off', 'PAY_2_no_card_use', 'PAY_2_payed_off',
    'PAY_3_no_card_use', 'PAY_3_payed_off', 'PAY_4_no_card_use', 'PAY_4_payed_off',
    'PAY_5_no_card_use', 'PAY_5_payed_off', 'PAY_6_no_card_use', 'PAY_6_payed_off'
]

disc_variables = ['SEX', 'EDUCATION', 'MARRIAGE']

cont_data = data_copy[cont_variables].values
disc_data = data_copy[disc_variables].values
print('Number of  variables:\t\t', cont_data.shape[1])
print('Number of categorical variables:\t', disc_data.shape[1])
target = data_copy['default.payment.next.month'].values

Number of  variables:		 32
Number of categorical variables:	 3


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

train_cont_data, test_cont_data, train_disc_data, test_disc_data, \
train_target, test_target = train_test_split(
    cont_data,
    disc_data,
    target,
    train_size=0.8,
    random_state=1000,
    shuffle=True)

def one_hot_coding(train, test):
    one_hot_coder = OneHotEncoder(handle_unknown='ignore', sparse_output = False)
    train = one_hot_coder.fit_transform(train)
    test = one_hot_coder.transform(test)
    return train, test

dev = pd.DataFrame(np.concatenate((train_cont_data, train_disc_data), axis=1))
val = pd.DataFrame(np.concatenate((test_cont_data, test_disc_data), axis =1))

train_disc_data, test_disc_data = one_hot_coding(train_disc_data,
                                                 test_disc_data)
print(train_disc_data.shape)
print(test_disc_data.shape)

train_data = np.concatenate((train_cont_data, train_disc_data), axis=1)
test_data = np.concatenate((test_cont_data, test_disc_data), axis=1)

(24000, 13)
(6000, 13)


In [8]:
from sklearn.preprocessing import StandardScaler
def normalize(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

train_data, test_data = normalize(train_data, test_data)

In [9]:
import pickle
model = pickle.load(open("model.pkl", 'rb'))

In [10]:
train_pred = model.predict_proba(train_data)[:, 1]
test_pred = model.predict_proba(test_data)[:, 1]


In [11]:
dev['actual'] = train_target
dev['pred'] = train_pred
dev.sort_values(by = 'pred')

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,actual,pred
2988,420000.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,62364.0,30165.0,...,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,0,0.022329
3668,480000.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,57732.0,55379.0,...,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,0,0.022378
5590,200000.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,132823.0,132700.0,...,1.0,1.0,1.0,1.0,1.0,2.0,5.0,1.0,0,0.022911
10340,360000.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,79837.0,27456.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,0,0.022955
21893,330000.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,18768.0,3736.0,...,1.0,1.0,1.0,1.0,1.0,2.0,5.0,1.0,0,0.023286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7885,10000.0,30.0,3.0,2.0,2.0,7.0,7.0,7.0,2300.0,2300.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1,0.930104
23948,10000.0,41.0,3.0,2.0,2.0,7.0,7.0,7.0,2400.0,2400.0,...,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1,0.933891
16613,20000.0,53.0,3.0,2.0,2.0,7.0,7.0,7.0,2255.0,2255.0,...,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1,0.935445
18324,10000.0,51.0,3.0,2.0,2.0,7.0,7.0,7.0,2400.0,2400.0,...,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,1,0.937855


In [12]:
def gini(df):
    deciles = pd.DataFrame((pd.qcut(df['pred'], 10, precision=16)).value_counts()).sort_values(by = 'pred', ascending=False)
    bad_count, good_count = [], []
    for index, row in deciles.iterrows():
        x = df[ (df['pred'] > index.left ) & (df['pred'] <= index.right)]
        bads = len(x[x['actual'] == 1])
        goods = len(x[x['actual'] == 0])
        bad_count.append(bads)
        good_count.append(goods)

    deciles['bads'] = bad_count
    deciles['goods'] = good_count
    deciles['total'] = deciles['goods'] + deciles['bads']
    deciles['cumulative bads'] = deciles['bads'].cumsum()
    deciles['cumulative goods'] = deciles['goods'].cumsum()
    deciles['cumulative total'] = deciles['total'].cumsum()
    deciles['bad rate'] = deciles['bads'] / deciles['total']
    deciles['cum bads%'] = 100* deciles['cumulative bads'] / deciles['bads'].sum()
    deciles['cum goods%'] = 100* deciles['cumulative goods'] / deciles['goods'].sum()
    deciles['cum all%'] = 100* deciles['cumulative total'] / deciles['total'].sum()
    deciles['AUC'] = (
        (deciles['cum bads%'] - deciles['cum bads%'].shift(1, fill_value=0)) *
        (deciles['cum goods%'] + deciles['cum goods%'].shift(1, fill_value=0)) / 2
    ) / 10000
    deciles['area_random'] = (
        (deciles['cum all%'] - deciles['cum all%'].shift(1, fill_value=0)) *
        (deciles['cum all%'] + deciles['cum all%'].shift(1, fill_value=0)) / 2
    ) / 10000
    deciles['B'] = deciles['area_random'] - deciles['AUC']
    deciles['gini'] = 2*deciles['B']

    return deciles, deciles['gini'].sum()

In [13]:
deciles_dev, gini_dev = gini(dev)

In [14]:
deciles_dev

,count,bads,goods,total,cumulative bads,cumulative goods,cumulative total,bad rate,cum bads%,cum goods%,cum all%,AUC,area_random,B,gini
pred,,,,,,,,,,,,,,,
"(0.5697423872159874, 0.9410441132328689]",2400,1774,626,2400,1774,626,2400,0.739167,33.459072,3.347952,10.0,0.005601,0.005,-0.000601,-0.001202
"(0.3159182452748875, 0.5697423872159874]",2400,1099,1301,2400,2873,1927,4800,0.457917,54.187099,10.305915,20.0,0.014151,0.015,0.000849,0.001698
"(0.2174790768460499, 0.3159182452748875]",2400,665,1735,2400,3538,3662,7200,0.277083,66.729536,19.584982,30.0,0.018745,0.025,0.006255,0.012510
"(0.1706438801739316, 0.2174790768460499]",2400,479,1921,2400,4017,5583,9600,0.199583,75.763863,29.858808,40.0,0.022335,0.035,0.012665,0.025331
"(0.1419315745485237, 0.1706438801739316]",2400,400,2000,2400,4417,7583,12000,0.166667,83.308186,40.555140,50.0,0.026561,0.045,0.018439,0.036877
"(0.1224831926191758, 0.1419315745485237]",2400,312,2088,2400,4729,9671,14400,0.130000,89.192757,51.722109,60.0,0.027151,0.055,0.027849,0.055699
"(0.1045202641425382, 0.1224831926191758]",2400,237,2163,2400,4966,11834,16800,0.098750,93.662769,63.290191,70.0,0.025705,0.065,0.039295,0.078589
"(0.08717371967116402, 0.1045202641425382]",2400,184,2216,2400,5150,14050,19200,0.076667,97.133157,75.141726,80.0,0.024021,0.075,0.050979,0.101959
"(0.06953555816345341, 0.08717371967116402]",2400,90,2310,2400,5240,16360,21600,0.037500,98.830630,87.495989,90.0,0.013804,0.085,0.071196,0.142393


In [15]:
gini_dev

np.float64(0.6219285117909198)

In [16]:
val['actual'] = test_target
val['pred'] = test_pred

In [17]:
val1 = val[:3000]
val2 = val[3000:]

In [18]:
deciles_val1, gini_val1 = gini(val1)

In [19]:
deciles_val1

,count,bads,goods,total,cumulative bads,cumulative goods,cumulative total,bad rate,cum bads%,cum goods%,cum all%,AUC,area_random,B,gini
pred,,,,,,,,,,,,,,,
"(0.5808312651174422, 0.9207605058650382]",300,209,91,300,209,91,300,0.696667,31.428571,3.897216,10.0,0.006124,0.005,-0.001124,-0.002248
"(0.3161433578149054, 0.5808312651174422]",300,138,162,300,347,253,600,0.460000,52.180451,10.835118,20.0,0.015286,0.015,-0.000286,-0.000572
"(0.2190011089945017, 0.3161433578149054]",300,80,220,300,427,473,900,0.266667,64.210526,20.256959,30.0,0.018702,0.025,0.006298,0.012596
"(0.1720268670971104, 0.2190011089945017]",300,62,238,300,489,711,1200,0.206667,73.533835,30.449679,40.0,0.023638,0.035,0.011362,0.022725
"(0.1422683892016613, 0.1720268670971104]",300,48,252,300,537,963,1500,0.160000,80.751880,41.241970,50.0,0.025874,0.045,0.019126,0.038253
"(0.1222118545079949, 0.1422683892016613]",300,42,258,300,579,1221,1800,0.140000,87.067669,52.291221,60.0,0.029537,0.055,0.025463,0.050926
"(0.1050514902931163, 0.1222118545079949]",300,37,263,300,616,1484,2100,0.123333,92.631579,63.554604,70.0,0.032228,0.065,0.032772,0.065544
"(0.08915899087673176, 0.1050514902931163]",300,27,273,300,643,1757,2400,0.090000,96.691729,75.246253,80.0,0.028178,0.075,0.046822,0.093645
"(0.06983109121038526, 0.08915899087673176]",300,11,289,300,654,2046,2700,0.036667,98.345865,87.623126,90.0,0.013470,0.085,0.071530,0.143059


In [20]:
gini_val1

np.float64(0.5828919193057591)

In [21]:
deciles_val2, gini_val2 = gini(val2)

In [22]:
gini_val2

np.float64(0.5915588875230131)

In [24]:
def psi_calc(val):
        max_prob=[]
        total_acc=[]
        total_val=[]
        for index, row in deciles_dev.iterrows():
                max_prob.append(index.right)
                total_acc.append(row['total'])
        psi = pd.DataFrame()
        psi['max_prob'] = max_prob
        psi['total_acc'] = total_acc
        psi['acc%'] = 100*(psi['total_acc'] / psi['total_acc'].sum())


        psi = psi.sort_values(by='max_prob')
        psi['max_prob1'] = psi['max_prob'].shift(1, fill_value=0)
        for index, row in psi.iterrows():
                x = val[ (val['pred'] > row['max_prob1'] ) & (val['pred'] <= row['max_prob'])]
                total = len(x)
                total_val.append(total)
        psi['total_val']=total_val
        psi['val%'] = 100*(psi['total_val'] / psi['total_val'].sum())

        psi['PSI'] = (
        (psi['acc%'] - psi['val%']) *
        np.log(psi['acc%'] / psi['val%'])
        ).replace([np.inf, -np.inf, np.nan], 0)
        total_psi = psi['PSI'].sum()
        print(psi)
        print(f'Total PSI: {total_psi}')
psi_calc(val1)

   max_prob  total_acc  acc%  max_prob1  total_val       val%       PSI
9  0.069536     2400.0  10.0   0.000000        295   9.833333  0.002801
8  0.087174     2400.0  10.0   0.069536        263   8.766667  0.162342
7  0.104520     2400.0  10.0   0.087174        333  11.100000  0.114796
6  0.122483     2400.0  10.0   0.104520        316  10.533333  0.027712
5  0.141932     2400.0  10.0   0.122483        291   9.700000  0.009138
4  0.170644     2400.0  10.0   0.141932        294   9.800000  0.004041
3  0.217479     2400.0  10.0   0.170644        297   9.900000  0.001005
2  0.315918     2400.0  10.0   0.217479        310  10.333333  0.010930
1  0.569742     2400.0  10.0   0.315918        291   9.700000  0.009138
0  0.941044     2400.0  10.0   0.569742        310  10.333333  0.010930
Total PSI: 0.35283179115085445


In [25]:
psi_calc(val2)

   max_prob  total_acc  acc%  max_prob1  total_val       val%       PSI
9  0.069536     2400.0  10.0   0.000000        310  10.333333  0.010930
8  0.087174     2400.0  10.0   0.069536        311  10.366667  0.013204
7  0.104520     2400.0  10.0   0.087174        293   9.766667  0.005509
6  0.122483     2400.0  10.0   0.104520        287   9.566667  0.019197
5  0.141932     2400.0  10.0   0.122483        295   9.833333  0.002801
4  0.170644     2400.0  10.0   0.141932        292   9.733333  0.007208
3  0.217479     2400.0  10.0   0.170644        307  10.233333  0.005382
2  0.315918     2400.0  10.0   0.217479        285   9.500000  0.025647
1  0.569742     2400.0  10.0   0.315918        323  10.766667  0.056634
0  0.941044     2400.0  10.0   0.569742        297   9.900000  0.001005
Total PSI: 0.14751547625724076
